In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd

import settings as conf
import metadata
from utils import is_number, chunker
from results.multixcan import MXPhenoInfo, MXPhenoResults

# fastENLOC reading functions

In [3]:
with open(conf.FASTENLOC_GTEX_TISSUES_FILE, 'r') as f:
    FASTENLOC_TISSUES_NAMES = set([x.strip() for x in f.readlines()])

In [4]:
# create summary of files
_path = os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], '**/*.enloc.enrich.out')
display(_path)
all_fastenloc_results_files = glob(_path)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/fastenloc/rapid_gwas_project/**/*.enloc.enrich.out'

In [5]:
len(all_fastenloc_results_files)

198401

In [6]:
PHENO_CODES = set(metadata.RAPID_GWAS_PHENO_INFO.index)

In [7]:
len(PHENO_CODES)

4359

In [8]:
phenos = []
tissues = []

for f in all_fastenloc_results_files:
    f = os.path.basename(f)
    split = f.split('-')
    
    tiss = split[-1].split('.')[0]
    if tiss not in FASTENLOC_TISSUES_NAMES:
        tiss = '-'.join(split[-2:]).split('.')[0]
    
    phen = split[1]
    if phen not in PHENO_CODES:
        phen = '-'.join(split[1:3])
    
    phenos.append(phen)
    tissues.append(tiss)

results_summary = pd.DataFrame({'pheno': phenos, 'tissue': tissues, 'file': all_fastenloc_results_files})

In [9]:
display(results_summary.shape)
assert results_summary.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

(198401, 3)

In [10]:
results_summary.head()

,pheno,tissue,file
0,R60,Liver,/home/miltondp/projects/labs/hakyimlab/phenome...
1,R60,Adipose_Visceral_Omentum,/home/miltondp/projects/labs/hakyimlab/phenome...
2,R60,Brain_Hypothalamus,/home/miltondp/projects/labs/hakyimlab/phenome...
3,R60,Nerve_Tibial,/home/miltondp/projects/labs/hakyimlab/phenome...
4,R60,Brain_Cerebellum,/home/miltondp/projects/labs/hakyimlab/phenome...


In [11]:
results_summary['pheno'].unique().shape

(4049,)

In [12]:
assert np.all([t in PHENO_CODES for t in results_summary['pheno'].unique()])

In [13]:
assert results_summary['tissue'].unique().shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [14]:
assert np.all([t in FASTENLOC_TISSUES_NAMES for t in results_summary['tissue'].unique()])

In [15]:
_all_tissues_in_results = results_summary['tissue'].unique()
assert np.all([t in _all_tissues_in_results for t in FASTENLOC_TISSUES_NAMES])

In [16]:
assert len(all_fastenloc_results_files) == int(conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

In [17]:
def read_fe_enrich(filename):
    enrich_data = pd.read_csv(filename, sep='\s+', header=None)
    enrich_data = enrich_data.rename(columns={0: 'type', 1: 'alpha', 2: 'se'})
    
    alpha0_data = enrich_data[enrich_data['type'] == 'Intercept']
    assert alpha0_data.shape[0] == 1
    alpha0_data = alpha0_data.iloc[0].loc[['alpha', 'se']]\
        .rename({'alpha': 'alpha0', 'se': 'alpha0_se'}).astype(float)
    
    alpha1_data = enrich_data[enrich_data['type'] == 'Enrichment']
    assert alpha1_data.shape[0] == 1
    alpha1_data = alpha1_data.iloc[0].loc[['alpha', 'se']]\
        .rename({'alpha': 'alpha1', 'se': 'alpha1_se'}).astype(float)
    
    df = pd.concat([alpha0_data, alpha1_data])
    return df
    

def read_fe_enrich_pheno(pheno): 
    _tmp = results_summary[results_summary['pheno'] == pheno] 
    if _tmp.shape[0] == 0: 
        return None 
     
    _res = {}
#     _all_genes = set()
    for idx, _data in _tmp.iterrows():
        tissue_data = read_fe_enrich(_data.file).rename(_data.pheno)
        _res[_data.tissue] = tissue_data
#         _all_genes.update(tissue_data.index) 
    
    df = pd.DataFrame.from_dict(_res, orient='index')#.T#, index=_all_genes)#.fillna(-1).max(axis=1)
    df.index.rename('tissue', inplace=True)
    df = df.reset_index().assign(trait=pheno)
    return df[['tissue', 'trait', 'alpha0', 'alpha0_se', 'alpha1', 'alpha1_se']]

In [18]:
results_summary.iloc[1].loc['file']

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/fastenloc/rapid_gwas_project/R60/fastenloc-R60-Adipose_Visceral_Omentum.enloc.enrich.out'

In [19]:
read_fe_enrich(results_summary.iloc[1].loc['file'])

alpha0        -11.772
alpha0_se       0.351
alpha1        -12.101
alpha1_se    1116.560
dtype: float64

In [20]:
read_fe_enrich_pheno('22617_9219').head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
0,Adipose_Subcutaneous,22617_9219,-14.575,0.877,-6.094,1040.506
1,Adipose_Visceral_Omentum,22617_9219,-14.395,0.721,5.625,217.299
2,Adrenal_Gland,22617_9219,-14.504,0.778,4.155,395.173
3,Artery_Aorta,22617_9219,-14.256,0.662,-10.846,2070.036
4,Artery_Coronary,22617_9219,-14.351,0.728,-4.269,1800.014


### Testing

In [21]:
# testing
t = read_fe_enrich(os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], 'J15/fastenloc-J15-Whole_Blood.enloc.enrich.out'))

In [22]:
assert t['alpha0'] == -11.752
assert t['alpha0_se'] == 0.350

assert t['alpha1'] == -11.286
assert t['alpha1_se'] == 764.307

In [23]:
t = read_fe_enrich_pheno('J15')

In [24]:
assert t.shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [25]:
t.head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
0,Adipose_Subcutaneous,J15,-11.725,0.347,-12.109,995.299
1,Adipose_Visceral_Omentum,J15,-11.747,0.349,-11.061,877.891
2,Adrenal_Gland,J15,-11.741,0.349,-11.018,1024.396
3,Artery_Aorta,J15,-11.730,0.348,-12.016,1029.679
4,Artery_Coronary,J15,-11.736,0.348,-11.979,1336.767


In [26]:
_t_tmp = t[t['tissue'] == 'Adipose_Visceral_Omentum'].iloc[0]
assert _t_tmp['tissue'] == 'Adipose_Visceral_Omentum'
assert _t_tmp['trait'] == 'J15'
assert _t_tmp['alpha0'] == -11.747
assert _t_tmp['alpha0_se'].round(3) == 0.349
assert _t_tmp['alpha1'] == -11.061
assert _t_tmp['alpha1_se'].round(3) == 877.891

_t_tmp = t[t['tissue'] == 'Artery_Coronary'].iloc[0]
assert _t_tmp['tissue'] == 'Artery_Coronary'
assert _t_tmp['trait'] == 'J15'
assert _t_tmp['alpha0'].round(3) == -11.736
assert _t_tmp['alpha0_se'].round(3) == 0.348
assert _t_tmp['alpha1'].round(3) == -11.979
assert _t_tmp['alpha1_se'].round(3) == 1336.767

# Get Rapid GWAS phenotypes

In [27]:
all_smultixcan_files = glob(os.path.join(conf.SMULTIXCAN_RESULTS_DIR['RapidGWASProject'], '*.tsv.gz'))
all_smultixcan_phenotypes = [MXPhenoResults(p) for p in all_smultixcan_files]

In [28]:
assert len(all_smultixcan_files) == len(all_smultixcan_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

In [29]:
all_smultixcan_files[:5]

['/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/smultixcan/rapid_gwas_project/smultixcan_20002_1267_ccn30.tsv.gz',
 '/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/smultixcan/rapid_gwas_project/smultixcan_C64_ccn30.tsv.gz',
 '/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/smultixcan/rapid_gwas_project/smultixcan_L12_SCARCONDITIONS_ccn30.tsv.gz',
 '/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/smultixcan/rapid_gwas_project/smultixcan_F5_BEHAVE_ccn30.tsv.gz',
 '/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/smultixcan/rapid_gwas_project/smultixcan_GOUT_ccn30.tsv.gz']

# Read all results

In [30]:
def _get_combined_results(phenos):
    all_res = [read_fe_enrich_pheno(pheno.pheno_info.pheno_code) for pheno in phenos]
    return pd.concat(all_res, ignore_index=True)

In [31]:
# testing
_tmp = _get_combined_results(all_smultixcan_phenotypes[:4])
assert _tmp.shape[0] == int(4 * 49)

In [32]:
_pending = read_fe_enrich_pheno('C_MYELOID-LEUKAEMIA')

In [33]:
_pending.head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
0,Adipose_Subcutaneous,C_MYELOID-LEUKAEMIA,-12.915,0.469,-11.482,1327.573
1,Adipose_Visceral_Omentum,C_MYELOID-LEUKAEMIA,-12.883,0.464,-11.767,1643.732
2,Adrenal_Gland,C_MYELOID-LEUKAEMIA,-12.913,0.468,-11.487,1752.318
3,Artery_Aorta,C_MYELOID-LEUKAEMIA,-12.983,0.482,-11.046,1233.831
4,Artery_Coronary,C_MYELOID-LEUKAEMIA,-12.897,0.467,-11.662,2048.158


In [34]:
def _run_all(phenotype_chunks, n_jobs=conf.N_JOBS_HIGH):
    all_results = []
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.append(res)

    return all_results

In [35]:
# phenotype_chunks = chunker(all_smultixcan_phenotypes[:5] + all_extra_phenotypes[:5], 2)
phenotype_chunks = chunker(all_smultixcan_phenotypes, 200)

In [36]:
all_results = _run_all(phenotype_chunks)

## Save as DataFrame

In [37]:
fastenloc_genes_associations = pd.concat(all_results, ignore_index=True)

display(fastenloc_genes_associations.shape)
display(fastenloc_genes_associations.head())

assert fastenloc_genes_associations.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

(198401, 6)

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
0,Adipose_Subcutaneous,20002_1274,-11.993,0.371,-11.876,1039.978
1,Adipose_Visceral_Omentum,20002_1274,-12.001,0.372,-11.836,1106.017
2,Adrenal_Gland,20002_1274,-11.998,0.372,-11.055,1182.881
3,Artery_Aorta,20002_1274,-11.992,0.372,-11.058,1019.484
4,Artery_Coronary,20002_1274,-12.004,0.372,-10.258,1115.669


In [38]:
assert fastenloc_genes_associations['tissue'].unique().shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [39]:
assert fastenloc_genes_associations['trait'].unique().shape[0] == conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject']

In [40]:
assert fastenloc_genes_associations.shape == fastenloc_genes_associations.dropna().shape

In [41]:
# check columns data type
_tmp = fastenloc_genes_associations.dtypes.value_counts()
display(_tmp)
assert _tmp.shape[0] == 2

float64    4
object     2
dtype: int64

In [42]:
fastenloc_genes_associations.isna().sum().sum()

0

In [43]:
# some testing
t = fastenloc_genes_associations[fastenloc_genes_associations['trait'] == '22601_12253140']

In [44]:
t.head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
133721,Adipose_Subcutaneous,22601_12253140,-10.779,0.274,-12.415,722.794
133722,Adipose_Visceral_Omentum,22601_12253140,-10.783,0.274,-12.281,776.066
133723,Adrenal_Gland,22601_12253140,-10.781,0.274,-12.428,959.603
133724,Artery_Aorta,22601_12253140,-10.780,0.274,-12.458,796.822
133725,Artery_Coronary,22601_12253140,-10.783,0.274,-12.448,1042.841


In [45]:
tt = t[t['tissue'] == 'Brain_Frontal_Cortex_BA9'].iloc[0]
assert tt.loc['alpha0'].round(3) == -10.781
assert tt.loc['alpha0_se'].round(3) == 0.274
assert tt.loc['alpha1'].round(3) == -12.584
assert tt.loc['alpha1_se'] == 1050.572

In [46]:
tt = t[t['tissue'] == 'Artery_Coronary'].iloc[0]
assert tt.loc['alpha0'].round(3) == -10.783
assert tt.loc['alpha0_se'].round(3) == 0.274
assert tt.loc['alpha1'].round(3) == -12.448
assert tt.loc['alpha1_se'].round(3) == 1042.841

### Save

In [47]:
os.makedirs(conf.ANALYSES_DIR, exist_ok=True)

In [48]:
# Save
fastenloc_genes_associations_filename = os.path.join(conf.ANALYSES_DIR, f'fastenloc-alpha1-stats.pkl.xz')
display(fastenloc_genes_associations_filename)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/analyses/fastenloc-alpha1-stats.pkl.xz'

In [49]:
fastenloc_genes_associations.to_pickle(fastenloc_genes_associations_filename)

### Save in text format

In [50]:
output_file = os.path.join(conf.ANALYSES_DIR, 'fastenloc-alpha1-stats.tsv.gz')
display(output_file)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/analyses/fastenloc-alpha1-stats.tsv.gz'

In [51]:
fastenloc_genes_associations.to_csv(output_file, index=False, sep='\t', float_format='%.3f')

In [52]:
# test "for publication" file
_tmp = pd.read_csv(output_file, sep='\t')

In [53]:
display(_tmp.shape)
assert _tmp.shape == fastenloc_genes_associations.shape

(198401, 6)

In [54]:
_tmp.head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se
0,Adipose_Subcutaneous,20002_1274,-11.993,0.371,-11.876,1039.978
1,Adipose_Visceral_Omentum,20002_1274,-12.001,0.372,-11.836,1106.017
2,Adrenal_Gland,20002_1274,-11.998,0.372,-11.055,1182.881
3,Artery_Aorta,20002_1274,-11.992,0.372,-11.058,1019.484
4,Artery_Coronary,20002_1274,-12.004,0.372,-10.258,1115.669


In [55]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [56]:
assert not _tmp.isna().any().any()

In [57]:
assert fastenloc_genes_associations.equals(_tmp)

# Apply alpha1 shrinkage

In [58]:
input_file = output_file
display(input_file)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/analyses/fastenloc-alpha1-stats.tsv.gz'

In [59]:
output_file = os.path.splitext(input_file)[0]
output_file = os.path.splitext(output_file)[0]
output_file += '-shrinkage.tsv'
display(output_file)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/analyses/fastenloc-alpha1-stats-shrinkage.tsv'

In [69]:
!Rscript ../extras/fastenloc/apply_alpha1_shrinkage.r $input_file $output_file 

                tissue      trait  alpha0 alpha0_se  alpha1 alpha1_se
1 Adipose_Subcutaneous 20002_1274 -11.993     0.371 -11.876  1039.978
     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-0.189282 -0.000010 -0.000004  0.510731 -0.000001  7.340497 
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.005446 0.999997 0.999999 0.887533 1.000000 1.000000 
[1] 198401      8


In [70]:
_tmp = pd.read_csv(output_file, sep='\s+', header=None)
_tmp = _tmp.rename(columns={
    0: 'tissue',
    1: 'trait',
    2: 'alpha0',
    3: 'alpha0_se',
    4: 'alpha1',
    5: 'alpha1_se',
    6: 'alpha1_shrinkage',
    7: 'alpha1_se_shrinkage',
})

In [71]:
_tmp.to_csv(output_file, sep='\t', index=False)

## Testing

In [72]:
# test "for publication" file
_tmp = pd.read_csv(output_file, sep='\t')

In [73]:
display(_tmp.shape)
assert _tmp.shape[0] == fastenloc_genes_associations.shape[0]

(198401, 8)

In [74]:
_tmp.head()

,tissue,trait,alpha0,alpha0_se,alpha1,alpha1_se,alpha1_shrinkage,alpha1_se_shrinkage
0,Adipose_Subcutaneous,20002_1274,-11.993,0.371,-11.876,1039.978,-0.000011,1.0
1,Adipose_Visceral_Omentum,20002_1274,-12.001,0.372,-11.836,1106.017,-0.000010,1.0
2,Adrenal_Gland,20002_1274,-11.998,0.372,-11.055,1182.881,-0.000008,1.0
3,Artery_Aorta,20002_1274,-11.992,0.372,-11.058,1019.484,-0.000011,1.0
4,Artery_Coronary,20002_1274,-12.004,0.372,-10.258,1115.669,-0.000008,1.0


In [75]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [76]:
assert not _tmp.isna().any().any()

In [77]:
assert np.array_equal(fastenloc_genes_associations.round(3), _tmp.iloc[:, 0:6].round(3))